In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

### Importing H2O

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [2]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\Dopefish\Anaconda3\envs\TF2\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Dopefish\AppData\Local\Temp\tmpv72mzq_y
  JVM stdout: C:\Users\Dopefish\AppData\Local\Temp\tmpv72mzq_y\h2o_Dopefish_started_from_python.out
  JVM stderr: C:\Users\Dopefish\AppData\Local\Temp\tmpv72mzq_y\h2o_Dopefish_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_Dopefish_k1jd6p
H2O cluster total nodes:,1
H2O cluster free memory:,3.535 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


### Loading Data

In [3]:
train=pd.read_csv(os.path.join(os.getcwd(),'train.csv.zip'))
test=pd.read_csv(os.path.join(os.getcwd(),'test.csv.zip'))
#train = h2o.import_file(os.path.join(os.getcwd(),'train.csv.zip'))
#test = h2o.import_file(os.path.join(os.getcwd(),'test.csv.zip'))

In [4]:
#class_weights={1: 0.37062, 2: 0.49657,3:0.05947,4:0.018,5:0.018,6:0.018,7:0.018}
class_weights={1: 0.37053, 2: 0.49657,3:0.05947,4:0.00106,5:0.01287,6:0.02698,7:0.03238}
list_weight=compute_class_weight(class_weights,np.unique(train['Cover_Type']),train['Cover_Type'])

In [5]:
def bal_acc2(y_true,y_pred):
    #weights should be provided as list of weights for each class to sum up to 1
    temp1=np.unique(y_true,return_counts=True)[1]
    temp2=confusion_matrix(y_true,y_pred)
    acc_bal=np.sum(np.diag(temp2)*(temp1/np.sum(temp1))/temp1)
    weights=[0.37053, 0.49657, 0.05947, 0.00106, 0.01287, 0.02698, 0.03238]
    acc_weig=np.sum(np.diag(temp2)*weights/temp1)
    return (acc_weig)

In [6]:
from sklearn.metrics import fbeta_score, make_scorer
my_score = make_scorer(bal_acc2)

In [7]:
#columns
target_col='Cover_Type'
soil_type=[x for x in train.columns if 'Soil' in x]
wild_type=[x for x in train.columns if 'Wilderness' in x]
vert_dist=['Vertical_Distance_To_Hydrology']
hor_dist=['Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Roadways',  'Horizontal_Distance_To_Fire_Points']
hill_feat=['Hillshade_9am', 'Hillshade_Noon','Hillshade_3pm']
other_feat=['Elevation', 'Aspect', 'Slope']

In [8]:
soil_names = {
0: 'Cathedral family - Rock outcrop complex, extremely stony.',
1: 'Vanet - Ratake families complex, very stony.',
2: 'Haploborolis - Rock outcrop complex, rubbly.',
3: 'Ratake family - Rock outcrop complex, rubbly.',
4: 'Vanet family - Rock outcrop complex complex, rubbly.',
5: 'Vanet - Wetmore families - Rock outcrop complex, stony.',
6: 'Gothic family.',
7: 'Supervisor - Limber families complex.',
8: 'Troutville family, very stony.',
9: 'Bullwark - Catamount families - Rock outcrop complex, rubbly.',
10: 'Bullwark - Catamount families - Rock land complex, rubbly.',
11: 'Legault family - Rock land complex, stony.',
12: 'Catamount family - Rock land - Bullwark family complex, rubbly.',
13: 'Pachic Argiborolis - Aquolis complex.',
14: 'unspecified in the USFS Soil and ELU Survey.',
15: 'Cryaquolls - Cryoborolis complex.',
16: 'Gateview family - Cryaquolls complex.',
17: 'Rogert family, very stony.',
18: 'Typic Cryaquolls - Borohemists complex.',
19: 'Typic Cryaquepts - Typic Cryaquolls complex.',
20: 'Typic Cryaquolls - Leighcan family, till substratum complex.',
21: 'Leighcan family, tibbll substratum, extremely bouldery.',
22: 'Leighcan family, till substratum - Typic Cryaquolls complex.',
23: 'Leighcan family, extremely stony.',
24: 'Leighcan family, warm, extremely stony.',
25: 'Granile - Catamount families complex, very stony.',
26: 'Leighcan family, warm - Rock outcrop complex, extremely stony.',
27: 'Leighcan family - Rock outcrop complex, extremely stony.',
28: 'Como - Legault families complex, extremely stony.',
29: 'Como family - Rock land - Legault family complex, extremely stony.',
30: 'Leighcan - Catamount families complex, extremely stony.',
31: 'Catamount family - Rock outcrop - Leighcan family complex, extremely stony.',
32: 'Leighcan - Catamount families - Rock outcrop complex, extremely stony.',
33: 'Cryorthents - Rock land complex, extremely stony.',
34: 'Cryumbrepts - Rock outcrop - Cryaquepts complex.',
35: 'Bross family - Rock land - Cryumbrepts complex, extremely stony.',
36: 'Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.',
37: 'Leighcan - Moran families - Cryaquolls complex, extremely stony.',
38: 'Moran family - Cryorthents - Leighcan family complex, extremely stony.',
39: 'Moran family - Cryorthents - Rock land complex, extremely stony.',}

In [9]:
def add_cat(df,soil_type,wild_type,soil_names,hill_feat):
    #first add categories
    df['soil_cat']=np.dot(df[soil_type],np.arange(len(soil_type)).reshape((len(soil_type),1)))
    df['wild_cat']=np.dot(df[wild_type],np.arange(len(wild_type)).reshape((len(wild_type),1)))
    df['soil_wild_cat']=df['soil_cat']*10+df['wild_cat']
    #next add names
    df["soil_name"]=df['soil_cat'].map(soil_names)
    #add some text features
    #check how stony is the soil
    df['stone_cat']=0
    df['stone_cat']+=df["soil_name"].str.contains(', rubbly')*1
    df['stone_cat']+=df["soil_name"].str.contains(', stony')*2
    df['stone_cat']+=df["soil_name"].str.contains('very stony')*3
    df['stone_cat']+=df["soil_name"].str.contains('extremely stony')*4
    df['wild_stone']=df['stone_cat']*10+df['wild_cat']    
    #check if Rock is in name
    df['Rock_cat']=df["soil_name"].str.contains('Rock outcrop')*1+df["soil_name"].str.contains('Rock land')*2
    df['if_Leighcan']=df["soil_name"].str.contains('Leighcan')*1
    df['if_Cryaquolls']=df["soil_name"].str.contains('Cryaquolls')*1
    df['if_Catamount']=df["soil_name"].str.contains('Catamount')*1
    df['if_Cryorthents']=df["soil_name"].str.contains('Cryorthents')*1    
    #relative distances
    df['Hyd_2_Road']=df['Horizontal_Distance_To_Hydrology']/(df['Horizontal_Distance_To_Roadways']+df['Horizontal_Distance_To_Hydrology']+0.0001)
    df['Hyd_2_Fire']=df['Horizontal_Distance_To_Hydrology']/(df['Horizontal_Distance_To_Fire_Points']+df['Horizontal_Distance_To_Hydrology']+0.0001)
    df['Fire_2_Road']=df['Horizontal_Distance_To_Fire_Points']/(df['Horizontal_Distance_To_Roadways']+df['Horizontal_Distance_To_Fire_Points']+0.0001)
    #add average shade
    df['av_shade']=(df[hill_feat[0]]+df[hill_feat[1]]+df[hill_feat[2]])/3
    df['dif_shade']=(df[hill_feat[0]]-df[hill_feat[2]])
    #add sin of Aspect and slope
    df['Sin_Aspect'] = np.sin(np.radians(df['Aspect']))
    df['Sin_Slope'] = np.sin(np.radians(df['Slope']))
    df["world_side"]=np.logical_and(df['Aspect']>90,df['Aspect']<=180)*1+np.logical_and(df['Aspect']>180,df['Aspect']<=270)*2+np.logical_and(df['Aspect']>270,df['Aspect']<=360)*3
    #below water
    df['if_below']=(df['Vertical_Distance_To_Hydrology']<0)*1
    df['Vertical_Distance_To_Hydrology']=np.abs(df['Vertical_Distance_To_Hydrology'])
    #convert to sqrt
    
    return df

In [10]:
train_new=add_cat(train,soil_type,wild_type,soil_names,hill_feat)
train_new['weights']=(train_new['Cover_Type']>2)*1+(train_new['Cover_Type']<3)*2

In [11]:
#below are features to use for traning
learn_cols=['Elevation',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'soil_cat', 'wild_cat', 'stone_cat', 'Hyd_2_Road', 'Hyd_2_Fire', 'Fire_2_Road', 'av_shade',
       'Sin_Aspect', 'Sin_Slope','dif_shade',"world_side",'wild_stone','soil_wild_cat','if_below', 'Rock_cat', 'if_Leighcan',
       'if_Cryaquolls', 'if_Catamount', 'if_Cryorthents']

### Convert Data to H2O

In [24]:
train_h2o=h2o.H2OFrame(train_new[learn_cols+['Cover_Type','weights']])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
#train_h2o["Cover_Type"].describe()

In [26]:
#Drop Id columns
# Make all categorical features as categorical
#NOTE SUPER IMPORTANT OTHERWISE IT WILL NOT UNDERSTAND THAT IT IS CATEGORICAL PROBLEM AND WILL USE REGRESSION!!!
train_h2o['Cover_Type'] = train_h2o['Cover_Type'].asfactor()
train_h2o['soil_cat'] = train_h2o['soil_cat'].asfactor()
train_h2o['wild_cat'] = train_h2o['wild_cat'].asfactor()
train_h2o['stone_cat'] = train_h2o['stone_cat'].asfactor()
train_h2o['world_side'] = train_h2o['world_side'].asfactor()
train_h2o['wild_stone'] = train_h2o['wild_stone'].asfactor()
train_h2o['soil_wild_cat'] = train_h2o['soil_wild_cat'].asfactor()
train_h2o['Rock_cat'] = train_h2o['Rock_cat'].asfactor()

#Predictor Columns
x_col = learn_cols
y_col = 'Cover_Type'

#Split data into training and validation
#d = train_h2o.split_frame(ratios = [0.8], seed = 42)
#hf_train = d[0] # using 80% for training
#hf_valid = d[1] # rest 20% for testing

### Setting the learning and fitting

For the AutoML function, we just specify how long we want to train for and we’re set. I aslo limit the number of models to 30, set number of kfolds to 5 and make the model to save the cross_validation_predictions

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

- x: A list/vector of predictor column names or indexes. This argument only needs to be specified if the user wants to exclude columns from the set of predictors. If all columns (other than the response) should be used in prediction, then this does not need to be set.
- validation_frame: This argument is ignored unless nfolds == 0, in which a validation frame can be specified and used for early stopping of individual models and early stopping of the grid searches (unless max_models or max_runtime_secs overrides metric-based early stopping). By default and when nfolds > 1, cross-validation metrics will be used for early stopping and thus validation_frame will be ignored.
- leaderboard_frame: This argument allows the user to specify a particular data frame to use to score and rank models on the leaderboard. This frame will not be used for anything besides leaderboard scoring. If a leaderboard frame is not specified by the user, then the leaderboard will use cross-validation metrics instead, or if cross-validation is turned off by setting nfolds = 0, then a leaderboard frame will be generated automatically from the training frame.
- blending_frame: Specifies a frame to be used for computing the predictions that serve as the training frame for the Stacked Ensemble models metalearner. If provided, all Stacked Ensembles produced by AutoML will be trained using Blending (a.k.a. Holdout Stacking) instead of the default Stacking method based on cross-validation.
- fold_column: Specifies a column with cross-validation fold index assignment per observation. This is used to override the default, randomized, 5-fold cross-validation scheme for individual models in the AutoML run.
- weights_column: Specifies a column with observation weights. Giving some observation a weight of zero is equivalent to excluding it from the dataset; giving an observation a relative weight of 2 is equivalent to repeating that row twice. Negative weights are not allowed.
- ignored_columns: (Optional, Python only) Specify the column or columns (as a list/vector) to be excluded from the model. This is the converse of the x argument.

In [52]:
aml = H2OAutoML(max_models=50, max_runtime_secs=3600, verbosity='info',nfolds=5,keep_cross_validation_predictions=True)
aml.train(x = x_col, y = y_col,weights_column='weights', training_frame = train_h2o)

AutoML progress: |
01:30:19.654: Project: automl_py_301_sid_8890
01:30:19.655: AutoML job created: 2019.09.16 01:30:19.654
01:30:19.655: AutoML: XGBoost is not available; skipping it.
01:30:19.655: Disabling Algo: XGBoost as requested by the user.
01:30:19.655: Build control seed: -1 (random)
01:30:19.655: training frame: Frame key: automl_training_py_301_sid_8890    cols: 29    rows: 15120  chunks: 1    size: 1024096  checksum: -710655351287482984
01:30:19.655: validation frame: NULL
01:30:19.655: leaderboard frame: NULL
01:30:19.655: response column: Cover_Type
01:30:19.655: fold column: null
01:30:19.655: weights column: [15120,1.0/1.285714285714286/2.0, 14.8 KB, {/127.0.0.1:54321:0:}]
01:30:19.656: Setting stopping tolerance adaptively based on the training frame: 0.008132500607904443
01:30:19.660: AutoML build started: 2019.09.16 01:30:19.660
01:30:19.660: AutoML: starting GLM hyperparameter search

███████████████████
01:34:39.950: New leader: GBM_4_AutoML_20190916_013019, mean_p

### Check the models

Once the model is trained, you can access the Leaderboard. The leader model is stored at aml.leader and the leaderboard is stored at aml.leaderboard The leaderboard stores the snapshot of the top models.

In [53]:
print(aml.leaderboard)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_AutoML_20190916_013019,0.11746,0.332595,0.309322,0.0956801
GBM_4_AutoML_20190916_013019,0.119444,0.428445,0.341029,0.116301
StackedEnsemble_AllModels_AutoML_20190916_003446,0.119577,0.335605,0.311312,0.0969153
GBM_grid_1_AutoML_20190916_013019_model_7,0.120503,0.41858,0.341691,0.116753
StackedEnsemble_BestOfFamily_AutoML_20190916_013019,0.120503,0.341712,0.313851,0.0985022
GBM_3_AutoML_20190916_013019,0.120899,0.406715,0.341182,0.116405
GBM_4_AutoML_20190916_003446,0.121164,0.417297,0.341731,0.11678
StackedEnsemble_BestOfFamily_AutoML_20190916_003446,0.123082,0.343468,0.315141,0.0993137
GBM_3_AutoML_20190916_003446,0.123611,0.404348,0.342094,0.117028
GBM_2_AutoML_20190916_003446,0.124405,0.404496,0.343905,0.118271


after 900 sec
StackedEnsemble_AllModels_AutoML_20190916_003446 	0.119577	0.335605	0.311312	0.0969153
GBM_4_AutoML_20190916_003446 	0.121164	0.417297	0.341731	0.11678
StackedEnsemble_BestOfFamily_AutoML_20190916_003446	0.123082	0.343468	0.315141	0.0993137
GBM_3_AutoML_20190916_003446 	0.123611	0.404348	0.342094	0.117028 

after 3600
StackedEnsemble_AllModels_AutoML_20190916_013019 	0.11746 	0.332595	0.309322	0.0956801
GBM_4_AutoML_20190916_013019 	0.119444	0.428445	0.341029	0.116301
StackedEnsemble_AllModels_AutoML_20190916_003446 	0.119577	0.335605	0.311312	0.0969153

In [54]:
print(aml.leader)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190916_013019

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.001671627565611271
RMSE: 0.040885542256539426

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.0956801183048794
RMSE: 0.3093220300995055



### get the individual models results

Note: here we will access the cross validation results;
.cross_validation_holdout_predictions() - gives the cross validation results and .cross_validation_predictions() gives the result of each cross validation

In [55]:
#save the models names
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

In [56]:
for model in model_ids:
    temp_model=h2o.get_model(model)
    temp_prediction=temp_model.cross_validation_holdout_predictions()
    if temp_prediction is None:
        pass
    else:
        temp_prediction=temp_prediction.as_data_frame()['predict']
        print('model ',model, "accuracy ", accuracy_score(train_new['Cover_Type'],temp_prediction),'bal accur',bal_acc2(train_new['Cover_Type'],temp_prediction))

model  GBM_4_AutoML_20190916_013019 accuracy  0.8805555555555555 bal accur 0.7762913379629628
model  GBM_grid_1_AutoML_20190916_013019_model_7 accuracy  0.8794973544973544 bal accur 0.7745515370370372
model  GBM_3_AutoML_20190916_013019 accuracy  0.879100529100529 bal accur 0.7755452962962962
model  GBM_4_AutoML_20190916_003446 accuracy  0.8788359788359789 bal accur 0.7740087870370371
model  GBM_3_AutoML_20190916_003446 accuracy  0.8763888888888889 bal accur 0.7701307546296297
model  GBM_2_AutoML_20190916_003446 accuracy  0.8755952380952381 bal accur 0.7725390277777777
model  GBM_2_AutoML_20190916_013019 accuracy  0.8751322751322751 bal accur 0.7662723796296296
model  GBM_5_AutoML_20190916_013019 accuracy  0.871031746031746 bal accur 0.7682720092592593
model  GBM_5_AutoML_20190916_003446 accuracy  0.8702380952380953 bal accur 0.7643194722222224
model  GBM_grid_1_AutoML_20190916_013019_model_9 accuracy  0.8698412698412699 bal accur 0.7562264305555557
model  GBM_1_AutoML_20190916_003446 

### see the weights of the ensemble model - it may hint which models to grab

Nope as meta is just weighted avarage

In [41]:
ensemble=h2o.get_model('StackedEnsemble_BestOfFamily_AutoML_20190916_003446')
#ensemble.coef_norm #for metalearner to see the weight of each meta model
#ensemble.std_coef_plot() #to see the plot

### now I will take 3 best models - one best on accuracy, best on balanced accuracy, best other type of model

In [59]:
temp=h2o.get_model('GBM_4_AutoML_20190916_013019').cross_validation_holdout_predictions().as_data_frame()['predict']

In [60]:
confusion_matrix(train_new['Cover_Type'],temp)

array([[1680,  347,    2,    0,   25,    3,  103],
       [ 337, 1605,   47,    0,  122,   39,   10],
       [   0,   14, 1823,   77,   19,  227,    0],
       [   0,    0,   27, 2111,    0,   22,    0],
       [   3,   47,   29,    0, 2063,   18,    0],
       [   1,   29,  142,   41,   11, 1936,    0],
       [  57,    6,    1,    0,    0,    0, 2096]], dtype=int64)

Save the cross-validation of leaders

In [61]:
h2o.get_model('GBM_4_AutoML_20190916_013019').cross_validation_holdout_predictions().as_data_frame().to_csv('h20_1_train_prob.csv')
h2o.get_model('GBM_3_AutoML_20190916_013019').cross_validation_holdout_predictions().as_data_frame().to_csv('h20_2_train_prob.csv')
h2o.get_model('GBM_grid_1_AutoML_20190916_013019_model_7').cross_validation_holdout_predictions().as_data_frame().to_csv('h20_3_train_prob.csv')

### Predicting the test

In [62]:
#load the data
test_new=add_cat(test,soil_type,wild_type,soil_names,hill_feat)
test_h2o=h2o.H2OFrame(test_new[learn_cols])
test_h2o['soil_cat'] = test_h2o['soil_cat'].asfactor()
test_h2o['wild_cat'] = test_h2o['wild_cat'].asfactor()
test_h2o['stone_cat'] = test_h2o['stone_cat'].asfactor()
test_h2o['world_side'] = test_h2o['world_side'].asfactor()
test_h2o['wild_stone'] = test_h2o['wild_stone'].asfactor()
test_h2o['soil_wild_cat'] = test_h2o['soil_wild_cat'].asfactor()
test_h2o['Rock_cat'] = test_h2o['Rock_cat'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [63]:
#save the predictions
h2o.get_model('GBM_4_AutoML_20190916_013019').predict(test_h2o).as_data_frame().to_csv('h20_1_test_prob.csv')
h2o.get_model('GBM_3_AutoML_20190916_013019').predict(test_h2o).as_data_frame().to_csv('h20_2_test_prob.csv')
h2o.get_model('GBM_grid_1_AutoML_20190916_013019_model_7').predict(test_h2o).as_data_frame().to_csv('h20_3_test_prob.csv')

gbm prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\Dopefish\Anaconda3\envs\TF2\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset column 'soil_cat' has levels not trained on: [6, 14]
  warnings.warn(w)
C:\Users\Dopefish\Anaconda3\envs\TF2\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset column 'wild_stone' has levels not trained on: [31]
  warnings.warn(w)
C:\Users\Dopefish\Anaconda3\envs\TF2\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset column 'soil_wild_cat' has levels not trained on: [60, 143, 151, 171]
  warnings.warn(w)


gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_binary_classification_product_backorders.ipynb

saving models

In [64]:
h2o.save_model(h2o.get_model('GBM_4_AutoML_20190916_013019'), path = "./H2O/mpdel_1")
h2o.save_model(h2o.get_model('GBM_3_AutoML_20190916_013019'), path = "./H2O/mpdel_2")
h2o.save_model(h2o.get_model('GBM_grid_1_AutoML_20190916_013019_model_7'), path = "./H2O/mpdel_3")

'F:\\Coding\\courses\\Kaggle\\challange\\sub 1309\\H2O\\mpdel_3\\GBM_grid_1_AutoML_20190916_013019_model_7'

Submission

In [65]:
predict=aml.leader.predict(test_h2o).as_data_frame()

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [66]:
#making prediction of leader 
sub=pd.DataFrame(test.Id.values,columns=['ID'])
sub['Cover_Type']=predict['predict'].values.astype('int64')
sub.to_csv('h2O.csv',index=False)